In [1]:
#Import Libraries
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torchvision.utils import save_image
from PIL import Image

In [2]:
#Load the base model
model=models.vgg19(pretrained=True).features
#Assign the variable device
device=torch.device("cuda" if (torch.cuda.is_available()) else 'cpu')

C:\Users\raven\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\raven\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
#Define the function
def image_loader(path):
    image=Image.open(path)
    loader=transforms.Compose([transforms.Resize((512,512)), transforms.ToTensor()])
    image=loader(image).unsqueeze(0)
    return image.to(device,torch.float)

#Load the base photo and the style image
original_image=image_loader('C:/Users/raven/Downloads/pics/Technological_Institute_of_the_Philippines_Quezon_City.jpg')
style_image=image_loader('C:/Users/raven/Downloads/pics/Bataan.jpg')

#Create the generated image 
generated_image=original_image.clone().requires_grad_(True)

In [4]:
#Define the class model
class VGG(nn.Module):
    def __init__(self):
        super(VGG,self).__init__()
        self.req_features= ['0','5','10','19','28'] 
        self.model=models.vgg19(pretrained=True).features[:29] 
   
    def forward(self,x):
        features=[]
        for layer_num,layer in enumerate(self.model):
            x=layer(x)
            if (str(layer_num) in self.req_features):
                features.append(x)
                
        return features

In [5]:
def calc_content_loss(gen_feat,orig_feat):
    content_l=torch.mean((gen_feat-orig_feat)**2)
    return content_l

In [6]:
def calc_style_loss(gen,style):
    batch_size,channel,height,width=gen.shape

    G=torch.mm(gen.view(channel,height*width),gen.view(channel,height*width).t())
    A=torch.mm(style.view(channel,height*width),style.view(channel,height*width).t())
        
    style_l=torch.mean((G-A)**2)
    return style_l

In [7]:
def calculate_loss(gen_features, orig_feautes, style_featues):
    style_loss=content_loss=0
    for gen,cont,style in zip(gen_features,orig_feautes,style_featues):
        content_loss+=calc_content_loss(gen,cont)
        style_loss+=calc_style_loss(gen,style)
    
    total_loss=alpha*content_loss + beta*style_loss 
    return total_loss

In [8]:
#Load the model
model=VGG().to(device).eval() 

#Initialize the parameters that are needed
epoch=7000
lr=0.004
alpha=8
beta=70


optimizer=optim.Adam([generated_image],lr=lr)

In [9]:
#Iterating for 1000 times
for e in range (epoch):
    gen_features=model(generated_image)
    orig_feautes=model(original_image)
    style_featues=model(style_image)
    

    total_loss=calculate_loss(gen_features, orig_feautes, style_featues)
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    if(e/100):
        print(total_loss)
        
        save_image(generated_image,"Bataan_Fernando_Amorsolo.png")    

tensor(7.9450e+09, grad_fn=<AddBackward0>)
tensor(6.8384e+09, grad_fn=<AddBackward0>)
tensor(6.1688e+09, grad_fn=<AddBackward0>)
tensor(5.7265e+09, grad_fn=<AddBackward0>)
tensor(5.4172e+09, grad_fn=<AddBackward0>)
tensor(5.1842e+09, grad_fn=<AddBackward0>)
tensor(4.9938e+09, grad_fn=<AddBackward0>)
tensor(4.8290e+09, grad_fn=<AddBackward0>)
tensor(4.6819e+09, grad_fn=<AddBackward0>)
tensor(4.5483e+09, grad_fn=<AddBackward0>)
tensor(4.4253e+09, grad_fn=<AddBackward0>)
tensor(4.3114e+09, grad_fn=<AddBackward0>)
tensor(4.2046e+09, grad_fn=<AddBackward0>)
tensor(4.1038e+09, grad_fn=<AddBackward0>)
tensor(4.0082e+09, grad_fn=<AddBackward0>)
tensor(3.9173e+09, grad_fn=<AddBackward0>)
tensor(3.8306e+09, grad_fn=<AddBackward0>)
tensor(3.7475e+09, grad_fn=<AddBackward0>)
tensor(3.6678e+09, grad_fn=<AddBackward0>)
tensor(3.5914e+09, grad_fn=<AddBackward0>)
tensor(3.5181e+09, grad_fn=<AddBackward0>)
tensor(3.4477e+09, grad_fn=<AddBackward0>)
tensor(3.3800e+09, grad_fn=<AddBackward0>)
tensor(3.31

tensor(1.6069e+08, grad_fn=<AddBackward0>)
tensor(1.5767e+08, grad_fn=<AddBackward0>)
tensor(1.5472e+08, grad_fn=<AddBackward0>)
tensor(1.5185e+08, grad_fn=<AddBackward0>)
tensor(1.4906e+08, grad_fn=<AddBackward0>)
tensor(1.4633e+08, grad_fn=<AddBackward0>)
tensor(1.4368e+08, grad_fn=<AddBackward0>)
tensor(1.4109e+08, grad_fn=<AddBackward0>)
tensor(1.3857e+08, grad_fn=<AddBackward0>)
tensor(1.3611e+08, grad_fn=<AddBackward0>)
tensor(1.3371e+08, grad_fn=<AddBackward0>)
tensor(1.3138e+08, grad_fn=<AddBackward0>)
tensor(1.2910e+08, grad_fn=<AddBackward0>)
tensor(1.2689e+08, grad_fn=<AddBackward0>)
tensor(1.2473e+08, grad_fn=<AddBackward0>)
tensor(1.2262e+08, grad_fn=<AddBackward0>)
tensor(1.2058e+08, grad_fn=<AddBackward0>)
tensor(1.1858e+08, grad_fn=<AddBackward0>)
tensor(1.1663e+08, grad_fn=<AddBackward0>)
tensor(1.1474e+08, grad_fn=<AddBackward0>)
tensor(1.1290e+08, grad_fn=<AddBackward0>)
tensor(1.1110e+08, grad_fn=<AddBackward0>)
tensor(1.0935e+08, grad_fn=<AddBackward0>)
tensor(1.07

tensor(32828518., grad_fn=<AddBackward0>)
tensor(32719256., grad_fn=<AddBackward0>)
tensor(32610712., grad_fn=<AddBackward0>)
tensor(32502948., grad_fn=<AddBackward0>)
tensor(32395944., grad_fn=<AddBackward0>)
tensor(32289662., grad_fn=<AddBackward0>)
tensor(32184138., grad_fn=<AddBackward0>)
tensor(32079318., grad_fn=<AddBackward0>)
tensor(31975182., grad_fn=<AddBackward0>)
tensor(31871768., grad_fn=<AddBackward0>)
tensor(31769038., grad_fn=<AddBackward0>)
tensor(31666976., grad_fn=<AddBackward0>)
tensor(31565596., grad_fn=<AddBackward0>)
tensor(31464888., grad_fn=<AddBackward0>)
tensor(31364798., grad_fn=<AddBackward0>)
tensor(31265356., grad_fn=<AddBackward0>)
tensor(31166570., grad_fn=<AddBackward0>)
tensor(31068354., grad_fn=<AddBackward0>)
tensor(30970794., grad_fn=<AddBackward0>)
tensor(30873870., grad_fn=<AddBackward0>)
tensor(30777514., grad_fn=<AddBackward0>)
tensor(30681786., grad_fn=<AddBackward0>)
tensor(30586692., grad_fn=<AddBackward0>)
tensor(30492172., grad_fn=<AddBack

tensor(19572130., grad_fn=<AddBackward0>)
tensor(19528898., grad_fn=<AddBackward0>)
tensor(19485798., grad_fn=<AddBackward0>)
tensor(19442852., grad_fn=<AddBackward0>)
tensor(19400044., grad_fn=<AddBackward0>)
tensor(19357410., grad_fn=<AddBackward0>)
tensor(19314908., grad_fn=<AddBackward0>)
tensor(19272550., grad_fn=<AddBackward0>)
tensor(19230352., grad_fn=<AddBackward0>)
tensor(19188294., grad_fn=<AddBackward0>)
tensor(19146358., grad_fn=<AddBackward0>)
tensor(19104570., grad_fn=<AddBackward0>)
tensor(19062922., grad_fn=<AddBackward0>)
tensor(19021398., grad_fn=<AddBackward0>)
tensor(18980028., grad_fn=<AddBackward0>)
tensor(18938822., grad_fn=<AddBackward0>)
tensor(18897748., grad_fn=<AddBackward0>)
tensor(18856780., grad_fn=<AddBackward0>)
tensor(18815954., grad_fn=<AddBackward0>)
tensor(18775288., grad_fn=<AddBackward0>)
tensor(18734726., grad_fn=<AddBackward0>)
tensor(18694304., grad_fn=<AddBackward0>)
tensor(18654024., grad_fn=<AddBackward0>)
tensor(18613856., grad_fn=<AddBack

tensor(13208348., grad_fn=<AddBackward0>)
tensor(13184039., grad_fn=<AddBackward0>)
tensor(13159802., grad_fn=<AddBackward0>)
tensor(13135611., grad_fn=<AddBackward0>)
tensor(13111494., grad_fn=<AddBackward0>)
tensor(13087431., grad_fn=<AddBackward0>)
tensor(13063441., grad_fn=<AddBackward0>)
tensor(13039505., grad_fn=<AddBackward0>)
tensor(13015630., grad_fn=<AddBackward0>)
tensor(12991810., grad_fn=<AddBackward0>)
tensor(12968064., grad_fn=<AddBackward0>)
tensor(12944374., grad_fn=<AddBackward0>)
tensor(12920755., grad_fn=<AddBackward0>)
tensor(12897195., grad_fn=<AddBackward0>)
tensor(12873660., grad_fn=<AddBackward0>)
tensor(12850221., grad_fn=<AddBackward0>)
tensor(12826835., grad_fn=<AddBackward0>)
tensor(12803495., grad_fn=<AddBackward0>)
tensor(12780233., grad_fn=<AddBackward0>)
tensor(12757023., grad_fn=<AddBackward0>)
tensor(12733868., grad_fn=<AddBackward0>)
tensor(12710771., grad_fn=<AddBackward0>)
tensor(12687742., grad_fn=<AddBackward0>)
tensor(12664777., grad_fn=<AddBack

KeyboardInterrupt: 